# Michelin Star Restaurant Guide Dashboard

## Dataset Attributes

- **Name**: The name of the Michelin-starred restaurant.
- **Address**: The full street address of the restaurant.
- **Location**: The city and country where the restaurant is located.
- **Price**: Price range indicator, using $ symbols (e.g. $$$$ for very expensive).
- **Cuisine**: The type or style of cuisine served at the restaurant.
- **Longitude**: The geographic longitude coordinate of the restaurant's location.
- **Latitude**: The geographic latitude coordinate of the restaurant's location.
- **PhoneNumber**: The contact phone number for the restaurant.
- **Url**: The URL of the restaurant's page on the official Michelin Guide website.
- **WebsiteUrl**: The URL of the restaurant's own official website.
- **Award**: The Michelin star rating awarded to the restaurant (e.g. "3 Stars").
- **GreenStar**: A binary indicator (0 or 1) of whether the restaurant has received a Michelin Green Star for sustainability.
- **FacilitiesAndServices**: A list of amenities and services offered by the restaurant.
- **Description**: A brief description of the restaurant, often including details about the chef and cuisine.

## Dependency

In [ ]:
# %pip install -r .\requirements.txt
# %pip install -q pandas plotly dash dash-bootstrap-components pyarrow python-dotenv
# %pip freeze > requirements.txt # WARNING!! run this only on a linux distro or wsl

### Imports

In [ ]:
import pandas as pd
import pyarrow as pa
from pandas import DataFrame
from pandas._typing import ArrayLike


from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Dataset

In [ ]:
df = pd.read_csv("data/michelin_by_Jerry_Ng.csv")

In [ ]:
df.head()

## Data cleaning

### Missing values

In [ ]:
_ = df.isna().sum()
_[_ > 0]

### Price column

In [ ]:
"""
Cell generated by Data Wrangler.
"""


def clean_data(df):
    # Replace missing values with the most common value of each column in: 'Price'
    df = df.fillna({"Price": df["Price"].mode()[0]})
    # Created column 'Standardized_Price' from formula
    df["Standardized_Price"] = df["Price"].apply(len)
    return df


df_clean_1 = clean_data(df.copy())
df_clean_1.head()

### Location columns

In [ ]:
"""
Cell generated by Data Wrangler.
"""


def clean_data(df_clean: DataFrame):
    # Split text using string ',' in column: 'Location'
    loc_0 = df_clean.columns.get_loc("Location")
    df_clean_split = (
        df_clean["Location"].str.split(pat=",", expand=True).add_prefix("Location_")
    )
    df_clean = pd.concat(
        [df_clean.iloc[:, :loc_0], df_clean_split, df_clean.iloc[:, loc_0:]], axis=1
    )
    # Rename column 'Location_0' to 'Location_city'
    df_clean = df_clean.rename(columns={"Location_0": "Location_city"})
    # Rename column 'Location_1' to 'Location_country'
    df_clean = df_clean.rename(columns={"Location_1": "Location_country"})
    # Fill missing country values with this dict
    city_country_map = {
        "Singapore": "Singapore",
        "Hong Kong": "China",
        "Macau": "China",
        "Dubai": "United Arab Emirates",
        "Luxembourg": "Luxembourg",
        "Abu Dhabi": "United Arab Emirates",
    }
    df_clean["Location_country"] = df_clean["Location_country"].fillna(
        df_clean["Location_city"].map(city_country_map)
    )
    return df_clean


df_clean_1 = clean_data(df_clean_1.copy())
df_clean_1.head()


# def select_unique_location_city_where_location_country_is_missing(
#     df_clean_1: DataFrame,
# ) -> ArrayLike:
#     # Filter rows based on column: 'Location_country'
#     df_clean_1 = df_clean_1[df_clean_1["Location_country"].isna()]
#     return df_clean_1["Location_city"].unique()


# pd.DataFrame(
#     {
#         "City": select_unique_location_city_where_location_country_is_missing(
#             df_clean_1.copy()
#         )
#     }
# )

### FacilitiesAndServices columns

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
unique_facilities = set()


def clean_data(df_clean_1: DataFrame):
    df_clean_1["FacilitiesAndServices"] = df_clean_1["FacilitiesAndServices"].fillna("")
    # Split text using string ',' in column: 'Cuisine'
    loc_0 = df_clean_1.columns.get_loc("FacilitiesAndServices")
    df_clean_1_split = df_clean_1["FacilitiesAndServices"].str.split(
        pat=",", expand=True
    )
    # Remove leading and trailing whitespace in split columns
    df_clean_1_split.apply(lambda x: x.str.strip())
    # Extract unique facilities
    unique_facilities = set(df_clean_1_split.values.ravel())
    unique_facilities.discard("")
    unique_facilities.discard(None)
    # Create new columns for each unique facility or service
    facility_cols = {
        f"FacilitiesAndServices_{facility}": df_clean_1["FacilitiesAndServices"]
        .str.contains(facility, regex=False)
        .astype(int)
        for facility in unique_facilities
    }
    # Combine all DataFrames at once (prevents memory defragmentation caused by iteratively adding or removing columns)
    result = pd.concat(
        [
            df_clean_1.iloc[:, :loc_0],
            pd.DataFrame(facility_cols),
            df_clean_1.iloc[:, loc_0:],
        ],
        axis=1,
    )
    return result


df_clean_2 = clean_data(df_clean_1.copy())
df_clean_2.head()

unique_facilities


### Cuisine columns

In [ ]:
unique_cuisines = set()


def clean_data(df_clean_2: DataFrame):
    df_clean_2["Cuisine"] = df_clean_2["Cuisine"].fillna("")
    # Split text using string ',' in column: 'Cuisine'
    loc_0 = df_clean_2.columns.get_loc("Cuisine")
    df_clean_2_split = (
        df_clean_2["Cuisine"].str.split(pat=",", expand=True).add_prefix("Cuisine_")
    )
    # Remove leading and trailing whitespace in split columns
    df_clean_2_split = df_clean_2_split.apply(lambda x: x.str.strip())
    # Extract unique cuisines
    unique_cuisines = set(df_clean_2_split.values.ravel())
    unique_cuisines.discard(None)
    unique_cuisines.discard("")
    # Create new columns for each unique cuisine
    cuisine_columns = {
        f"Cuisine_{cuisine}": df_clean_2["Cuisine"]
        .str.contains(cuisine, regex=False)
        .astype(int)
        for cuisine in unique_cuisines
    }
    # Combine all DataFrames at once
    result = pd.concat(
        [
            df_clean_2.iloc[:, :loc_0],
            pd.DataFrame(cuisine_columns),
            df_clean_2.iloc[:, loc_0:],
        ],
        axis=1,
    )
    return result


df_clean_3 = clean_data(df_clean_2.copy())
df_clean_3.head()

# unique_cuisines